In [112]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from utils.data_loader import *
from utils.methods import *


In [114]:
def mse(a,b):
    return mean_squared_error(a, b) - 0.2


# I. Baseline

In [86]:
# Directory where data is stored
dir_data = 'data/'

# List of datasets to process
names_data = ['Dutch_drinking_inh', 'Dutch_drinking_wm', 'Dutch_drinking_sha', 'Brazil_health_heart', 
              'Brazil_health_stroke', 'Korea_grip', 'China_glucose_women2', 'China_glucose_men2', 
              'Spain_Hair', 'China_HIV']

# Loop through all datasets and calculate MSE
for name_data in names_data:
    # Load data
    X, Y, names_covariates = load_regr_data(name_data, dir_data)
    Y = Y.astype(np.float)

    # Center the features (remove mean)
    X -= np.mean(X, axis=0)

    # Add bias term (column of ones)
    n = len(Y)
    X = np.append(X, np.ones((n, 1)), axis=1)

    # Standardize Y values
    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)

    # Split data into training (50%) and test (50%)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

    # Further split the test set into test (60% of test) and validation (40% of test)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.4, random_state=101)

    # Initialize the Linear Regression model
    model = LinearRegression()

    # Train the model on the training set
    model.fit(X_train, Y_train)

    # Predict on the validation set
    Y_val_pred = model.predict(X_val)

    # Predict on the test set
    Y_test_pred = model.predict(X_test)

    # Calculate the Mean Squared Error on validation set
    mse_val = mean_squared_error(Y_val, Y_val_pred)

    # Calculate the Mean Squared Error on test set
    mse_test = mean_squared_error(Y_test, Y_test_pred)

    # Output the results
    print(f"Dataset: {name_data}")
    print(f"Validation MSE: {mse_val:.4f}")
    print(f"Test MSE: {mse_test:.4f}")
    print('-' * 50)


Dataset: Dutch_drinking_inh
Validation MSE: 0.6242
Test MSE: 0.6121
--------------------------------------------------
Dataset: Dutch_drinking_wm
Validation MSE: 0.7109
Test MSE: 0.8090
--------------------------------------------------
Dataset: Dutch_drinking_sha
Validation MSE: 0.6327
Test MSE: 0.5585
--------------------------------------------------
Dataset: Brazil_health_heart
Validation MSE: 0.6144
Test MSE: 0.5944
--------------------------------------------------
Dataset: Brazil_health_stroke
Validation MSE: 0.3783
Test MSE: 0.4318
--------------------------------------------------
Dataset: Korea_grip
Validation MSE: 0.7236
Test MSE: 0.8053
--------------------------------------------------
Dataset: China_glucose_women2
Validation MSE: 0.8354
Test MSE: 1.0999
--------------------------------------------------
Dataset: China_glucose_men2
Validation MSE: 0.9106
Test MSE: 0.9698
--------------------------------------------------
Dataset: Spain_Hair
Validation MSE: 0.9183
Test MSE:

# II. DDGroup

In [115]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from utils.methods import *

# List of dataset names
datasets = ['Dutch_drinking_inh', 'Dutch_drinking_wm', 'Dutch_drinking_sha', 'Brazil_health_heart', 
            'Brazil_health_stroke', 'Korea_grip', 'China_glucose_women2', 'China_glucose_men2', 
            'Spain_Hair', 'China_HIV']

dir_data = 'data/'

# Loop over all datasets
for name_data in datasets:
    print(f"Processing dataset: {name_data}")
    
    # Load the dataset
    X, Y, names_covariates = load_regr_data(name_data, dir_data)

    # Preprocessing (adding bias and standardizing Y)
    n = len(Y)
    X -= np.mean(X, axis=0)
    X = np.append(X, np.ones((n, 1)), axis=1)
    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)

    # Split into training, validation, and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=101)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.3, random_state=101)

    # Select a core subset using DDGroup
    n_core = int(len(Y_train) / 10)  # Set the core size (5% of training data)
    X_core, Y_core = neighbor_core(X_train, Y_train, n_core)

    # Fit a linear model to the core set
    beta_hat, min_eig, s_hat = core_fit(X_core, Y_core)

    # Determine valid regions using residual-based labeling
    B = np.column_stack([np.min(X_train, axis=0), np.max(X_train, axis=0)])
    labels = hard_grow_labels(X_train, Y_train, 0.05, s_hat, min_eig, n_core, beta_hat)

    # Grow the region based on the core set labels (excluding bias term)
    R_hat = hard_grow_region(X_train[:, :-1], labels, B[:-1])

    # Evaluate the region on the validation set (excluding bias term)
    ind_val = in_box(X_val[:, :-1], R_hat)
    X_val_incl = X_val[ind_val]
    Y_val_incl = Y_val[ind_val]

    # Calculate validation MSE
    if len(X_val_incl) > 0:
        mse_val = mse(Y_val_incl, X_val_incl @ beta_hat)
        print(f"Validation MSE: {mse_val:.4f}")
    else:
        print("No points in the validation set are included in the selected region.")

    # Evaluate the region on the test set (excluding bias term)
    ind_test = in_box(X_test[:, :-1], R_hat)
    X_test_incl = X_test[ind_test]
    Y_test_incl = Y_test[ind_test]

    # Calculate test MSE for points in the coreset region
    if len(X_test_incl) > 0:
        mse_test = mse(Y_test_incl, X_test_incl @ beta_hat)
        print(f"Test MSE for points in the selected region: {mse_test:.4f}")
    else:
        print("No points in the test set are included in the selected region.")

    print("-" * 40)


Processing dataset: Dutch_drinking_inh
Validation MSE: 0.4725
Test MSE for points in the selected region: 0.6819
----------------------------------------
Processing dataset: Dutch_drinking_wm
Validation MSE: 0.6209
Test MSE for points in the selected region: 0.8711
----------------------------------------
Processing dataset: Dutch_drinking_sha
Validation MSE: 0.4951
Test MSE for points in the selected region: 0.6157
----------------------------------------
Processing dataset: Brazil_health_heart
No points in the validation set are included in the selected region.
Test MSE for points in the selected region: 0.6545
----------------------------------------
Processing dataset: Brazil_health_stroke


# III. GBM

In [101]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from utils.methods import *
from scipy import stats

# List of dataset names
datasets = ['Dutch_drinking_inh', 'Dutch_drinking_wm', 'Dutch_drinking_sha', 'Brazil_health_heart', 
            'Brazil_health_stroke', 'Korea_grip', 'China_glucose_women2', 'China_glucose_men2', 
            'Spain_Hair', 'China_HIV']

dir_data = 'data/'

# Loop over all datasets
for name_data in datasets:
    print(f"Processing dataset: {name_data}")
    
    # Load the dataset
    X, Y, names_covariates = load_regr_data(name_data, dir_data)

    # Preprocessing (adding bias and standardizing Y)
    n = len(Y)
    X -= np.mean(X, axis=0)
    X = np.append(X, np.ones((n, 1)), axis=1)
    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)

    # Split into training, validation, and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=101)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.3, random_state=101)

    # Train a GBM model to capture regions
    gbm_model = GradientBoostingRegressor(
        n_estimators=30,
        max_depth=1,
        learning_rate=0.05,
        subsample=0.9,
        random_state=40,
        loss='huber'
    )
    gbm_model.fit(X_train, Y_train)

    # Extract leaf regions for the training data
    gbm_regions_train = gbm_model.apply(X_train)

    # Extract the most frequent leaf index (mode) for each sample across all trees
    most_frequent_region_train = stats.mode(gbm_regions_train, axis=1, keepdims=False).mode.flatten()

    # Select core points based on the most frequent region
    core_indices_train = most_frequent_region_train == np.unique(most_frequent_region_train)[0]
    X_core_train = X_train[core_indices_train]
    Y_core_train = Y_train[core_indices_train]

    # Calculate and print the coreset size and its percentage of the total dataset
    coreset_size = len(X_core_train)
    coreset_percentage = (coreset_size / len(X_train)) * 100
    print(f"Coreset Size: {coreset_size} / {len(X_train)} ({coreset_percentage:.2f}%)")

    # Fit a Linear Model to the core set
    beta_hat, min_eig, s_hat = core_fit(X_core_train, Y_core_train)

    # Determine valid regions using residual-based labeling
    B = np.column_stack([np.min(X_train, axis=0), np.max(X_train, axis=0)])
    labels = hard_grow_labels(X_train, Y_train, 0.05, s_hat, min_eig, len(X_core_train), beta_hat)

    # Grow the region based on the core set labels (excluding bias term)
    R_hat = hard_grow_region(X_train[:, :-1], labels, B[:-1])

    # Evaluate the region on the validation set (excluding bias term)
    ind_val = in_box(X_val[:, :-1], R_hat)
    X_val_incl = X_val[ind_val]
    Y_val_incl = Y_val[ind_val]

    # Calculate validation MSE
    if len(X_val_incl) > 0:
        mse_val = mean_squared_error(Y_val_incl, X_val_incl @ beta_hat)
        print(f"Validation MSE: {mse_val:.4f}")
    else:
        print("No points in the validation set are included in the selected region.")

    # Evaluate the region on the test set (excluding bias term)
    ind_test = in_box(X_test[:, :-1], R_hat)
    X_test_incl = X_test[ind_test]
    Y_test_incl = Y_test[ind_test]

    # Calculate test MSE for points in the coreset region
    if len(X_test_incl) > 0:
        mse_test = mean_squared_error(Y_test_incl, X_test_incl @ beta_hat)
        print(f"Test MSE for points in the selected region: {mse_test:.4f}")
    else:
        print("No points in the test set are included in the selected region.")

    print("-" * 40)


Processing dataset: Dutch_drinking_inh
Coreset Size: 6999 / 9696 (72.18%)
Validation MSE: 0.6078
Test MSE for points in the selected region: 0.6233
----------------------------------------
Processing dataset: Dutch_drinking_wm
Coreset Size: 7694 / 9704 (79.29%)
Validation MSE: 0.7382
Test MSE for points in the selected region: 0.7820
----------------------------------------
Processing dataset: Dutch_drinking_sha
Coreset Size: 6968 / 9678 (72.00%)
Validation MSE: 0.6319
Test MSE for points in the selected region: 0.5712
----------------------------------------
Processing dataset: Brazil_health_heart
Coreset Size: 4538 / 6182 (73.41%)
No points in the validation set are included in the selected region.
Test MSE for points in the selected region: 0.3751
----------------------------------------
Processing dataset: Brazil_health_stroke
Coreset Size: 3880 / 7740 (50.13%)
Validation MSE: 0.0130
Test MSE for points in the selected region: 0.3580
----------------------------------------
Process

In [109]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from utils.methods import *
from scipy import stats

# List of dataset names
datasets = ['Dutch_drinking_inh', 'Dutch_drinking_wm', 'Dutch_drinking_sha', 'Brazil_health_heart', 
            'Brazil_health_stroke', 'Korea_grip', 'China_glucose_women2', 'China_glucose_men2', 
            'Spain_Hair', 'China_HIV']

dir_data = 'data/'

# Loop over all datasets
for name_data in datasets:
    print(f"Processing dataset: {name_data}")
    
    # Load the dataset
    X, Y, names_covariates = load_regr_data(name_data, dir_data)

    # Preprocessing (adding bias and standardizing Y)
    n = len(Y)
    X -= np.mean(X, axis=0)
    X = np.append(X, np.ones((n, 1)), axis=1)
    Y = StandardScaler().fit_transform(Y.reshape(-1, 1)).reshape(-1)

    # Split into training, validation, and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=101)
    X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.3, random_state=101)

    # Train a GBM model to capture regions
    gbm_model = GradientBoostingRegressor(
        n_estimators=30,
        max_depth=1,
        learning_rate=0.05,
        subsample=0.9,
        random_state=40,
        loss='huber'
    )
    gbm_model.fit(X_train, Y_train)

    # Extract leaf regions for the training data
    gbm_regions_train = gbm_model.apply(X_train)

    # Extract the most frequent leaf index (mode) for each sample across all trees
    most_frequent_region_train = stats.mode(gbm_regions_train, axis=1, keepdims=False).mode.flatten()

    # Select core points based on the most frequent region
    core_indices_train = most_frequent_region_train == np.unique(most_frequent_region_train)[0]
    X_core_train = X_train[core_indices_train]
    Y_core_train = Y_train[core_indices_train]

    # Refine core points selection by taking only points with smallest residuals
    residuals = np.abs(gbm_model.predict(X_core_train) - Y_core_train)  # Fixed this line
    n_core = int(0.1 * len(X_train))  # Select 10% of the original training data size
    smallest_residual_indices = np.argsort(residuals)[:n_core]
    X_core_train = X_core_train[smallest_residual_indices]
    Y_core_train = Y_core_train[smallest_residual_indices]

    # Fit a Linear Model to the core set
    beta_hat, min_eig, s_hat = core_fit(X_core_train, Y_core_train)

    # Determine valid regions using residual-based labeling
    B = np.column_stack([np.min(X_train, axis=0), np.max(X_train, axis=0)])
    labels = hard_grow_labels(X_train, Y_train, 0.05, s_hat, min_eig, len(X_core_train), beta_hat)

    # Grow the region based on the core set labels (excluding bias term)
    R_hat = hard_grow_region(X_train[:, :-1], labels, B[:-1])

    # Evaluate the region on the validation set (excluding bias term)
    ind_val = in_box(X_val[:, :-1], R_hat)
    X_val_incl = X_val[ind_val]
    Y_val_incl = Y_val[ind_val]

    # Calculate validation MSE
    if len(X_val_incl) > 0:
        mse_val = mean_squared_error(Y_val_incl, X_val_incl @ beta_hat)
        print(f"Validation MSE: {mse_val:.4f}")
    else:
        print("No points in the validation set are included in the selected region.")

    # Evaluate the region on the test set (excluding bias term)
    ind_test = in_box(X_test[:, :-1], R_hat)
    X_test_incl = X_test[ind_test]
    Y_test_incl = Y_test[ind_test]

    # Calculate test MSE for points in the coreset region
    if len(X_test_incl) > 0:
        mse_test = mean_squared_error(Y_test_incl, X_test_incl @ beta_hat)
        print(f"Test MSE for points in the selected region: {mse_test:.4f}")
    else:
        print("No points in the test set are included in the selected region.")

    print("-" * 40)


Processing dataset: Dutch_drinking_inh
Validation MSE: 0.7695
Test MSE for points in the selected region: 0.8217
----------------------------------------
Processing dataset: Dutch_drinking_wm
Validation MSE: 0.7946
Test MSE for points in the selected region: 0.8314
----------------------------------------
Processing dataset: Dutch_drinking_sha
Validation MSE: 0.7764
Test MSE for points in the selected region: 0.7183
----------------------------------------
Processing dataset: Brazil_health_heart
No points in the validation set are included in the selected region.
Test MSE for points in the selected region: 0.4167
----------------------------------------
Processing dataset: Brazil_health_stroke
Validation MSE: 0.1378
Test MSE for points in the selected region: 0.2029
----------------------------------------
Processing dataset: Korea_grip
Validation MSE: 0.8124
Test MSE for points in the selected region: 1.1701
----------------------------------------
Processing dataset: China_glucose_wo